Практическая работа №4. Классификация. Вариант 19

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

BankChurners.csv - файл с данными.

	Бизнес-менеджер портфеля потребительских кредитных карт сталкивается с проблемой оттока клиентов. Они хотят проанализировать данные, чтобы выяснить причину этого, и использовать ее для прогнозирования клиентов, которые, скорее всего, уйдут.

Удаляю ненужные столбцы из data:

	CLIENTNUM - Номер клиента. Уникальный идентификатор клиента, владеющего учетной записью.

ПОЖАЛУЙСТА, ПРОИГНОРИРУЙТЕ ПОСЛЕДНИЕ 2 СТОЛБЦА (NAIVE BAYES CLAS…). Я ПРЕДЛАГАЮ СКОРЕЕ УДАЛИТЬ ЕГО, ПРЕЖДЕ ЧЕМ ЧТО-ЛИБО ДЕЛАТЬ** (c)

	Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1 	Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2

Далее меняю столбцы на дамми-перем
енные:

	Attrition_Flag - Переменная внутреннего события (активность клиента). Меняю на 1, если активна (Existing Customer), иначе на 0 (Attrited Customer).
	Gender - Переменная, отвечающая за пол. Меняю на: 0 - женский, 1 - мужской

В итоге выводим отредактированный набор данных data.

In [8]:
data = pd.read_csv("BankChurners.csv", delimiter=',')
data.drop("CLIENTNUM", axis=1, inplace=True)
data.drop("Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2", axis=1, inplace=True)
data.drop("Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", axis=1, inplace=True)
data["Attrition_Flag"] = np.where(data["Attrition_Flag"] == "Existing Customer", 0, 1)
data["Gender"] = np.where(data["Gender"] == 'F', 0, 1)

print(data)

       Attrition_Flag  Customer_Age  Gender  Dependent_count Education_Level  \
0                   0            45       1                3     High School   
1                   0            49       0                5        Graduate   
2                   0            51       1                3        Graduate   
3                   0            40       0                4     High School   
4                   0            40       1                3      Uneducated   
...               ...           ...     ...              ...             ...   
10122               0            50       1                2        Graduate   
10123               1            41       1                2         Unknown   
10124               1            44       0                1     High School   
10125               1            30       1                2        Graduate   
10126               1            43       0                2        Graduate   

      Marital_Status Income_Category Ca

Всё успешно отредатировалось. Удаляем все строки с NA полями.

In [9]:
data = data.dropna()

Ввожу остальные дамми-переменные, которые отвечают за уровень образование, брачный статус, категорию.

	Education_Level - Демографическая переменная - Образовательный ценз владельца учетной записи (пример: выпускник средней школы, колледжа и т.д.)
	Marital_Status - Демографическая переменная - Женат, не женат, разведен, неизвестен
	Card_Category - Параметр продукта - Тип карты (синяя, серебряная, золотая, платиновая)

In [10]:
data["isgraduate"] = np.where(data["Education_Level"]=="Graduate", 1, 0)
data["ishigh"] = np.where(data["Education_Level"]=="High School", 1, 0)
data["isunknowneduc"] = np.where(data["Education_Level"]=="Unknown", 1, 0)
data["ismarried"] = np.where(data["Marital_Status"]=="Married",1, 0)
data["issingle"] = np.where(data["Marital_Status"] =="Single", 1, 0)
data["isunknownmar"] = np.where(data["Marital_Status"] =="Unknown", 1, 0)
data["issilver"] = np.where(data["Card_Category"]=="Silver", 1, 0)
data["isblue"] = np.where(data["Card_Category"]=="Blue", 1, 0)
data["isgold"] = np.where(data["Card_Category"]=="Gold", 1, 0)
data["isplatinum"] = np.where(data["Card_Category"]=="Platinum", 1, 0)
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  int32  
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  int32  
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

Добавляю в набор данных дамми-перменные связанные с уровнем дохода. Разделяю на категории уровня дохода.

	Income_Category - Демографическая переменная - Категория годового дохода владельца счета (< $40 тыс., $40 тыс. - 60 тыс., $60 тыс. - $80 тыс., $80 тыс.-$120 тыс., >

In [11]:
data["<40k"] = np.where(data["Income_Category"]=="Less than $40K", 1, 0)
data["$40K - $60K"] = np.where(data["Income_Category"]=="$40K - $60K", 1, 0)
data["$60K - $80K"] = np.where(data["Income_Category"]=="$60K - $80K", 1, 0)
data["$80K-$120K"] = np.where(data["Income_Category"]=="$80K - $120K", 1, 0)
data["$120K+"] = np.where(data["Income_Category"]=="$120K +", 1, 0)


Удаляю из набора данных использованные столбцы, так как уже создал необходимые столбцы с дамми-переменными. Они больше не понадобятся для классификации.

In [12]:
data.drop(columns=["Education_Level", "Marital_Status", "Card_Category", "Income_Category"], axis=1, inplace=True)
print(data.info())

<bound method DataFrame.info of        Attrition_Flag  Customer_Age  Gender  Dependent_count  Months_on_book  \
0                   0            45       1                3              39   
1                   0            49       0                5              44   
2                   0            51       1                3              36   
3                   0            40       0                4              34   
4                   0            40       1                3              21   
...               ...           ...     ...              ...             ...   
10122               0            50       1                2              40   
10123               1            41       1                2              25   
10124               1            44       0                1              36   
10125               1            30       1                2              36   
10126               1            43       0                2              25   

       

Выделяю целевой признак Gender и удаляю его из данных, на основе которых будет обучаться классификатор.

In [13]:
Gender = data["Gender"]
X = data.drop("Gender", axis=1)

# Логистическая регрессия

Разделию набор данных на тестовую и обучающую выборку.

Строю классификатор LogisticRegression (логистическая регрессия) с нормой штрафа L2.
Алгоритм для использования в задаче оптимизации - lbfgs

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Gender, test_size=0.3)
classifier = LogisticRegression(random_state = 0, solver = 'lbfgs', multi_class = 'multinomial', penalty = 'l2', max_iter=1000)#, penalty = 'elasticnet', l1_ration = '0.5' #'saga' 
classifier.fit(X_train, Y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=0)

Оцениваю точность построенного классификатора логистической регрессии с помощью метрик precision, recall и F1 на тестовой выборке

In [18]:
y_pred = classifier.predict(X_test)
classifier.score(X_test, Y_test)

0.8733135899967095

In [20]:
TP = 0
FP = 0
FN = 0
TN = 0
for i in np.where(Y_test == 1)[0]:
    if y_pred[i] == 1:
        TP+=1
    else:
        FN+=1
for i in np.where(Y_test == 0)[0]:
    if y_pred[i] == 1:
        FP+=1
    else:
        TN+=1
print(TP,FP,FN, TN)
precision = TP / (TP + FP)
recall = TP/(TP + FN)
F1 = 2* (precision*recall)/(precision+recall)
print(precision, recall,F1)

1249 175 210 1405
0.8771067415730337 0.8560657984921178 0.8664585501214014


# Случайный лес

Строю классификатор Случайный Лес (Random Forest) из 100-300 (шаг 50) решающих деревьев и его оценка с помощью метрик accuracy, recall, f1, precision.

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators':[100,150,200,250,300],'max_features':['auto'],'max_depth': list(range(1,20))}
forest = GridSearchCV(estimator = RandomForestClassifier(),param_grid = param_grid,cv = 5, refit = True)
forest.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19],
                         'max_features': ['auto'],
                         'n_estimators': [100, 150, 200, 250, 300]})

In [26]:
forest.best_estimator_


RandomForestClassifier(max_depth=19, n_estimators=200)

С помощью GridSearchCV выясняю, что лучшим классификатором по числу деревьев является Random Forest из 200 решающих деревьев и глубиной 19.
Оцениваю точность построенного классификатора с помощью метрик accuracy, precision, recall и F1 на тестовой выборке.

In [27]:
print("accuracy:"+str(np.average(cross_val_score(forest.best_estimator_, X_test, Y_test, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(forest.best_estimator_, X_test, Y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(forest.best_estimator_, X_test, Y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(forest.best_estimator_, X_test, Y_test, scoring='recall'))))

accuracy:0.8953546345270095
f1:0.8835512453827515
precision:0.9202133712536826
recall:0.850560184531375


Повышаю точность. Иду далее строить случайный лес из больше чем 200 деревьев (уменьшать смысла нет, так как у классификаторов, состоящих из менее, чем 200 деревьев хуже точность)

С помощью GridSearchCV определяю, какой лес наилучший по количеству решающих деревьев Random Forest.

In [29]:
param_grid = {'n_estimators':list(range(200, 251, 10)),'max_features':['auto'],'max_depth': [19]} 
forest = GridSearchCV(estimator = RandomForestClassifier(),param_grid = param_grid,cv = 5, refit = True)
forest.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [19], 'max_features': ['auto'],
                         'n_estimators': [200, 210, 220, 230, 240, 250]})

In [31]:
forest.best_estimator_

RandomForestClassifier(max_depth=19, n_estimators=250)

In [32]:
param_grid = {'n_estimators':list(range(230, 271, 10)),'max_features':['auto'],'max_depth': [19]} 
forest = GridSearchCV(estimator = RandomForestClassifier(),param_grid = param_grid,cv = 5, refit = True)
forest.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [19], 'max_features': ['auto'],
                         'n_estimators': [230, 240, 250, 260, 270]})

In [33]:
forest.best_estimator_

RandomForestClassifier(max_depth=19, n_estimators=240)

In [35]:
param_grid = {'n_estimators':list(range(240, 255, 5)),'max_features':['auto'],'max_depth': [19]} 
forest = GridSearchCV(estimator = RandomForestClassifier(),param_grid = param_grid,cv = 5, refit = True)
forest.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [19], 'max_features': ['auto'],
                         'n_estimators': [240, 245, 250]})

In [36]:
forest.best_estimator_

RandomForestClassifier(max_depth=19, n_estimators=250)

In [37]:
param_grid = {'n_estimators':list(range(240, 255, 3)),'max_features':['auto'],'max_depth': [19]} 
forest = GridSearchCV(estimator = RandomForestClassifier(),param_grid = param_grid,cv = 5, refit = True)
forest.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [19], 'max_features': ['auto'],
                         'n_estimators': [240, 243, 246, 249, 252]})

In [ ]:
forest.best_estimator_

RandomForestClassifier(max_depth=19, n_estimators=243)

Я пришёл к лучшему результату для классификатора решающее дерево. Сравниваю его показатели с классификатором логистической регрессии:

Random Forest:
	accuracy:	0.8953546345270095
	f1:			0.8835512453827515
	precision:	0.9202133712536826
	recall:		0.850560184531375

Logistic Regression:
	accuracy:	0.8771067415730337
	f1:			0.8560657984921178
	precision:	0.8664585501214014
	recall:		0.8733135899967095

Из данных метрик заметно, что все значения метрик выше у Random Forest из 243 решающих деревьев чем у Logistic Regression.

Отсюда делаю вывод, что лучшим из классификаторов оказался Random Forest из 243 решающих деревьев, исходя из значений метрик: accuracy, f1, recall, precision.